In [1]:
import time
from io import StringIO
import requests
from bs4 import BeautifulSoup
import pandas as pd
years = list(range(2024, 2022, -1))


In [2]:
years

[2024, 2023]

In [3]:
all_matches = []

In [4]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [9]:


all_matches = []
for year in years:
    # Solicitar la página de clasificaciones
    data = requests.get(standings_url)
    time.sleep(3)  # Pausa después de cada solicitud
    soup = BeautifulSoup(data.text, "html.parser")
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

        # Solicitar los datos del equipo
        data = requests.get(team_url)
        time.sleep(3)  # Pausa después de cada solicitud
        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]
        print(matches)

        soup = BeautifulSoup(data.text, "html.parser")
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]

        # Solicitar los datos de disparos
        data = requests.get(f"https://fbref.com{links[0]}")
        time.sleep(3)  # Pausa después de cada solicitud
        shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]
        print(shooting)

        try:
            # Asegúrate de restablecer el índice de 'shooting' si tiene múltiples niveles
            if isinstance(shooting.columns, pd.MultiIndex):
                shooting.columns = shooting.columns.get_level_values(0)  # Usa solo el primer nivel de las columnas

            # Filtra columnas que contengan las palabras clave necesarias
            required_keywords = ["Date", "Dist", "FK", "PK", "PKatt"]
            shooting_filtered = shooting[
                [col for col in shooting.columns if any(keyword in col for keyword in required_keywords)]
            ]

            # Realiza el merge usando las columnas filtradas
            team_data = matches.merge(shooting_filtered, on="Date")
        except ValueError as e:
            print(f"Error al combinar datos para {team_name}: {e}")
            continue



        team_data = team_data[team_data["Comp"] == "Premier League"]

        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)

        # Mensaje indicando que se completó la tabla de un equipo
        print(f"Tabla completada para el equipo {team_name} en la temporada {year}.")


          Date   Time            Comp         Round  Day    Venue Result  GF  \
0   2020-09-21  20:15  Premier League   Matchweek 2  Mon     Away      W   3   
1   2020-09-24  19:45         EFL Cup   Third round  Thu     Home      W   2   
2   2020-09-27  16:30  Premier League   Matchweek 3  Sun     Home      L   2   
3   2020-09-30  19:00         EFL Cup  Fourth round  Wed     Away      W   3   
4   2020-10-03  17:30  Premier League   Matchweek 4  Sat     Away      D   1   
..         ...    ...             ...           ...  ...      ...    ...  ..   
56  2021-05-08  17:30  Premier League  Matchweek 35  Sat     Home      L   1   
57  2021-05-14  20:00  Premier League  Matchweek 36  Fri     Away      W   4   
58  2021-05-18  19:00  Premier League  Matchweek 37  Tue     Away      L   2   
59  2021-05-23  16:00  Premier League  Matchweek 38  Sun     Home      W   5   
60  2021-05-29  22:00    Champions Lg         Final  Sat  Neutral      L   0   

    GA        Opponent   xG  xGA  Poss 

KeyError: 'Date'

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import time

def fetch_team_urls(soup):
    """Extracts team URLs from the standings page."""
    links = [l.get("href") for l in soup.select('table.stats_table')[0].find_all('a')]
    return [f"https://fbref.com{l}" for l in links if '/squads/' in l]

def fetch_match_data(team_url):
    """Fetches match data for a team."""
    data = requests.get(team_url)
    matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]
    return matches

def fetch_shooting_data(team_url):
    """Fetches shooting data for a team."""
    soup = BeautifulSoup(requests.get(team_url).text, "html.parser")
    links = [l.get("href") for l in soup.find_all('a') if l and 'all_comps/shooting/' in l]
    if not links:
        return None
    shooting_data = requests.get(f"https://fbref.com{links[0]}")
    shooting = pd.read_html(StringIO(shooting_data.text), match="Shooting")[0]
    return shooting

def scrape_season_data(standings_url, season):
    """Scrapes data for a given season."""
    all_matches = []
    while standings_url:
        data = requests.get(standings_url)
        soup = BeautifulSoup(data.text, "html.parser")
        team_urls = fetch_team_urls(soup)
        for team_url in team_urls:
            try:
                team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
                matches = fetch_match_data(team_url)
                shooting = fetch_shooting_data(team_url)
                if shooting is not None:
                    team_data = matches.merge(shooting[["Date", "Dist", "FK", "PK", "PKatt"]], on="Date")
                    team_data = team_data[team_data["Comp"] == "Premier League"]
                    team_data["Season"] = season
                    team_data["Team"] = team_name
                    all_matches.append(team_data)
                    print(f"Procesado correctamente {team_url}: {e}")
            except Exception as e:
                print(f"Failed to process {team_url}: {e}")
            time.sleep(1)
        previous_season = soup.select_one("a.prev")
        standings_url = f"https://fbref.com{previous_season['href']}" if previous_season else None
    return pd.concat(all_matches, ignore_index=True) if all_matches else pd.DataFrame()

# Example Usage
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
season_data = scrape_season_data(standings_url, "2021-2022")
print(season_data.head())


IndexError: list index out of range

In [ ]:
team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
team_name


In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

In [ ]:
match_df.to_csv("matches.csv")